In [141]:
#import library
from skimage.metrics import structural_similarity
import cv2
import numpy as np
import pandas as pd
import statistics 

In [142]:
#function to count number of black pixel 
def count_black(image):
    white_count= np.sum(image == 255)
    non_white_count = np.sum( image != 255)
    
    return white_count , non_white_count

In [143]:
# to crop the image , x= stating x cordinate , y = starting y cordinate , n = total_len / no. of images
def crop_image(image ,x,y,n):
    crop_image=image[x*n : x*n + n , y*n : y*n +  n]
    return crop_image
    

In [144]:
#convert image into grayscale 
# note dimension of image has been concert into 500,500 we can change it as per our need
def preprocess(image):
    img1 = cv2.resize(image,(500 ,500), interpolation = cv2.INTER_AREA)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    return img1

In [145]:
# match no.of black pixwl in both image and return , result(dictionary with zeros and ones , zero if two image have differnt
 # black pixel , diff = difference btw black pixels in images , count= to no. of similar images )
def compare(image_detail , compare_image_detail):
    result={}
    diff={}
    count=0
    # note if we change dimension or  "n in crop image we have to change range of i and j according to that "
    for i in range(0,5):
        for j in range(0,5):
            diff[str(i)+str(j)]= abs(image_detail[str(i)+str(j)][1] - compare_image_detail[str(i)+str(j)][1])
     #note if we want are changing no. of split in image we should adjust " limit " accornding in below line 
     # note as per my observation a single line is equal to 200 count so decide limit according to that 
            if diff[str(i)+str(j)] < 100:
                result[str(i)+str(j)]=1
            else :
                result[str(i)+str(j)]=0
            if result[str(i)+str(j)]==1:
                count=count+1
    return result ,diff  ,count         

In [164]:
# will find the expected return and variance for next two days 
# input data = ohlc data , sample_data = list of dates on which similar pattern has been observed 
def mean(sample_data , data):
    ret =[]
    pos_ret=[]
    neg_ret=[]
    pos_count=0
    neg_count =0 
    postion_to_take =None
    for i in sample_data:
        ret.append(data['return'][i+1]+data['return'][i+2])
        if (data['return'][i+1]+data['return'][i+2])  >0 :
            pos_count = pos_count+1
            pos_ret.append(data['return'][i+1]+data['return'][i+2])
        else :
            neg_count=neg_count+1
            neg_ret.append(data['return'][i+1]+data['return'][i+2])
    mean = statistics.mean(ret)
    var = statistics.pstdev(ret)
    if pos_count> neg_count :
        postion_to_take = "long"
    else :
        postion_to_take = "short"
    return mean , var ,ret , pos_count/len(sample_data) , neg_count/len(sample_data) , pos_ret , neg_ret ,postion_to_take

In [147]:
# dict will store no. of black pixel in different segement of images
image_detail={}


In [148]:
# img to be search , converted into gray
img1 = preprocess(cv2.imread('image/Fig858.png'))

In [149]:
# break img1  into 25 small part and count no. of black pixel in each segment 
# if we want 100 or any other no. then change the image dimension from 500,500 to some other number and also change
# the range of i anf j in below code
for i in range(0,5):
    for j in range(0,5):
        image_detail[str(i)+str(j)]= count_black(crop_image(img1 , i ,j ,100))

In [150]:
# dict will store the deatils() when we will compare img1 and image in dataset
# please run this line aagin if you are comparing again 
diff_img = {}


In [154]:
#note plz run above line before ,,,,,think why to run ???
#note plz change the value 1300 according to no. of images you have 
for k in range(0,1300):
    img2 =preprocess(cv2.imread("image/Fig"+str(k)+".png"))
    compare_image_detail={}
# range of i and j should be change if we are changing split in image 
    for i in range(0,5):
        for j in range(0,5):
            compare_image_detail[str(i)+str(j)]= count_black(crop_image(img2 , i ,j ,100))
    diff_img[str(k)]=compare(image_detail,compare_image_detail)        
    

In [155]:
# will store the dates on which similar image has being observed
sample_data=[]

In [156]:
# note please change the value 1300 and 16 better to take it 15 (decide acc. to no. of data point you got )
# if ypu are getting more point change value to 18 or 17 in number of data points are less change it to 14 or 15
for k in range(0,1300):
     if diff_img[str(k)][2]>16:
            sample_data.append(k)
        

In [ ]:
cv2.imshow("img1",crop_image(img1,2,1,100))
cv2.imshow("img2",crop_image(img2,2,1,100))
cv2.waitKey(0)

In [157]:
#import data
data= pd.read_csv('SBIN.csv',index_col=0).dropna().copy()
data['return']=data['close'].pct_change()*100

In [158]:
df = pd.read_csv('SBIN.csv',index_col=0).dropna().copy()
df.index = pd.to_datetime(df.index)

In [165]:
 mean(sample_data , data)

(1.024424668645392,
 1.2284557836399472,
 [-0.1781564286780113,
  0.6945510419726797,
  2.8693901523454413,
  -1.0684157594400223,
  1.3839905814543085,
  0.4249093245289992,
  2.3543001926742857,
  1.7148282443054552],
 0.75,
 0.25,
 [0.6945510419726797,
  2.8693901523454413,
  1.3839905814543085,
  0.4249093245289992,
  2.3543001926742857,
  1.7148282443054552],
 [-0.1781564286780113, -1.0684157594400223],
 'long')